# fshafas Examples

In [ ]:

from typing import (Optional, List, Any, Tuple, TypeVar,
                    Tuple, Awaitable, Union, Callable, Dict)
import pandas as pd
from statistics import mean
from functools import reduce
from ipyleaflet import (Map, Marker, Icon, MarkerCluster, TileLayer, basemaps, basemap_to_tiles)
from fshafas import (HafasClient, Journey, Default_LocationsOptions, Default_JourneysOptions, Default_RadarOptions)
from fshafas.profiles import db_profile
from util import (to_location, to_locations, to_bounding_box, centroid)

## Locations

Retrieve location data of Hannover.

In [ ]:
with HafasClient(db_profile) as client:
    stops = await client.locations("Hannover", Default_LocationsOptions)
    locations = to_locations(stops)


In [ ]:
pd.DataFrame((locations), columns=["id", "name", "longitude", "latitude"])

## Journeys

Retrieve current journeys from Hamm to Hamburg.

In [ ]:
with HafasClient(db_profile) as client:
    journeys_result = await client.journeys("Hamm", "Hamburg", Default_JourneysOptions)
    journeys = journeys_result.journeys

Prepare data for DataFrame

In [ ]:
def to_legs_of_journey(journey: Journey, idx):
    return [dict(
                origin = to_location(leg.origin).name, 
                destination = to_location(leg.destination).name,
                departure = leg.departure,
                arrival = leg.arrival,
                id = idx,
                price = journey.price.amount if journey.price is not None else 0)
            for leg in journey.legs]

def reducer(acc: Tuple[List[Any], int], journey: Journey):
    return [acc[0] + to_legs_of_journey(journey, acc[1]), acc[1] + 1]

legs, _ = reduce(reducer, journeys, ([], 0))


In [ ]:
tuples = map(lambda l: (l["id"], l["price"], l["origin"]), legs)
index = pd.MultiIndex.from_tuples(tuples, names = ["id", "price", "origin"])

pd.DataFrame(legs, columns = ["destination",
             "departure", "arrival"], index = index)


## Radar

Retrieve all trains currently in an area between Münster and Hannover.

In [ ]:
with HafasClient(db_profile) as client:
    stops_1 = await client.locations("Münster", Default_LocationsOptions)
    stops_2 = await client.locations("Hannover", Default_LocationsOptions)

rect = to_bounding_box(stops_1, stops_2)

with HafasClient(db_profile) as client:
    opt = Default_RadarOptions
    opt.duration = 2400
    opt.frames = 10
    opt.products = client.productsOfMode(db_profile, "train")
    radar = await client.radar(rect, opt)


In [ ]:
data = [dict(
            trip_id = movement.trip_id,
            line = movement.line.name,
            longitude = movement.location.longitude,
            latitude = movement.location.latitude)
        for movement in radar.movements]

pd.DataFrame(data, columns=["trip_id", "line", "longitude", "latitude"])


Display train positions on a map.

In [ ]:
m = Map(basemap=basemaps.OpenStreetMap.Mapnik, center=centroid(radar.movements, rect), zoom=8)

m.add_layer(basemap_to_tiles(basemaps.OpenRailwayMap))

icon_url = 'https://raw.githubusercontent.com/Leaflet/Leaflet/main/dist/images/marker-icon.png'
icon = Icon(icon_url=icon_url, icon_size = [25, 40], icon_anchor=[22, 94])

markers = [Marker(location = (m.location.latitude, m.location.longitude), title = m.line.name,
                  icon = icon, draggable = False) for m in radar.movements]

marker_cluster = MarkerCluster(markers=markers)

m.add_layer(marker_cluster)

m
